# Test to see if gpu is open

In [12]:
if torch.cuda.is_available():
    print('GPU is available')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print("No GPU found")

GPU is available
Device name: NVIDIA GeForce RTX 3050 Ti Laptop GPU


# Imports

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from helper import (CelebCariDataset, CelebCariTestDataset, calculate_accuracy, save_gallery_to_json, create_gallery_embeddings, read_gallery_from_json, encode_embedding, write_predictions_to_json)
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd


In [14]:
# Define the dataset and dataloaders with data augmentation for the training set
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

# No augmentation for validation and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create dataset instances
train_dataset = CelebCariDataset(root_dir='./Project/train', transform=train_transform)
val_dataset = CelebCariDataset(root_dir='./Project/validation', transform=transform)
test_dataset = CelebCariTestDataset(root_dir='./Project/test', transform=transform)

# Create dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:

class MultiLabelModel(nn.Module):
    def __init__(self, num_identities, num_styles):
        super(MultiLabelModel, self).__init__()
        self.backbone = models.resnet18(pretrained=True)
        
        # Capture the number of features in the last layer of the backbone
        num_features = self.backbone.fc.in_features
        
        # Remove the final classification layer
        self.backbone.fc = nn.Identity()
        
        # Identity prediction head
        self.identity_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Linear(512, num_identities)
        )
        
        # Style prediction head
        self.style_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Linear(512, num_styles)
        )
        
    def forward(self, x):
        features = self.backbone(x)
        identity_logits = self.identity_head(features)
        style_logits = self.style_head(features)
        return features, identity_logits, style_logits

# Instantiate the model and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_identities = 12  # Replace with the actual number of identities in your dataset
num_styles = 6  # Replace with the actual number of styles in your dataset
model = MultiLabelModel(num_identities, num_styles).to(device)

# Define loss functions
identity_criterion = nn.CrossEntropyLoss()
style_criterion = nn.CrossEntropyLoss()

def multi_label_loss(identity_logits, style_logits, identity_labels, style_labels):
    identity_loss = identity_criterion(identity_logits, identity_labels)
    style_loss = style_criterion(style_logits, style_labels)
    return identity_loss + style_loss

# Define the optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


# Create a training dataset instance
train_dataset = CelebCariDataset(root_dir='./Project/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 25  # Adjust based on your needs

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, (identity_labels, style_labels) in train_loader:
        inputs = inputs.to(device)
        identity_labels = identity_labels.to(device)
        style_labels = style_labels.to(device)
        
        optimizer.zero_grad()
        features, identity_logits, style_logits = model(inputs)
        loss = multi_label_loss(identity_logits, style_logits, identity_labels, style_labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")
    scheduler.step()

# Validation
model.eval()
with torch.no_grad():
    # Implement validation accuracy calculation here
    pass

# Save gallery embeddings
gallery_embeddings = create_gallery_embeddings(model, train_dataset, device)
save_gallery_to_json(gallery_embeddings, 'gallery_embeddings.json')

# Create a test dataset instance
test_dataset = CelebCariTestDataset(root_dir='./Project/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load gallery embeddings
loaded_gallery_embeddings = read_gallery_from_json('gallery_embeddings.json')

# Make predictions
model.eval()
predictions = []
with torch.no_grad():
    for inputs, image_paths in test_loader:
        inputs = inputs.to(device)
        embeddings, identity_logits, style_logits = model(inputs)
        for embedding, image_path in zip(embeddings, image_paths):
            # Perform identity recognition using cosine similarity with gallery embeddings
            # Dummy style prediction for illustration; replace with actual prediction logic
            style_prediction = torch.argmax(style_logits, dim=1)
            predictions.append((image_path, embedding, style_prediction.item()))

# Write predictions to a JSON file
write_predictions_to_json(predictions, 'test_predictions.json')


d:\Yaşar\Yazılım\Neural-Networks-With-Pytorch\.conda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Yaşar\Yazılım\Neural-Networks-With-Pytorch\.conda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: too many values to unpack (expected 2)